In [1]:
from mmdet.apis import init_detector
import mmcv
from mmcv import Config


import copy
import os.path as osp

import numpy as np

from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset

from mmdet.apis import set_random_seed


import json

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import glob as _glob
import pandas as pd
import os
    
def glob(dir, pats, recursive=False):  # faster than match, python3 only
    pats = pats if isinstance(pats, (list, tuple)) else [pats]
    matches = []
    for pat in pats:
        matches += _glob.glob(os.path.join(dir, pat), recursive=recursive)
    return matches


In [3]:
#### load_annotations에서 뒤의 변수 받는거 custom dataset 에서는 이름을 바꿔도 되지만 아래에
#### configuration에서는 무조건 변수명을 ann_file로 받아야함
@DATASETS.register_module()
class Drive_dataset(CustomDataset):
    CLASSES=('car','bus','truck', 'special vehicle', 'motorcycle','bicycle','personal mobility','person','Traffic_light', 'Traffic_sign')


    def load_annotations(self, ann_file):
        
        CLASSES_dict = {'car' : 0 , 'bus' : 1, 'truck' : 2, 'special vehicle' : 3, 'motorcycle' : 4,'bicycle' : 5 ,'personal mobility' : 6 
                        ,'person' : 7 ,'Traffic_light' : 8, 'Traffic_sign' : 9}
        
        cat2label = {k: i for i, k in enumerate(self.CLASSES)}
        
        data_infos = []
        
        ls = pd.read_csv(ann_file, header = None)
        
        for idx,an in enumerate(ls.values):
            an=an[0]
            json_data = {}
            with open(an, "r") as json_file:
                json_data = json.load(json_file)
                
            ansplit = an.split('/')
            
            filename = ansplit[0] + '/' + ansplit[1] + '/' + 'images'+'/'+ json_data['image_name']
            
            width, height = json_data['image_size']

            data_info = dict(filename=filename, width=width, height=height)

            gt_bboxes = []
            gt_labels = []

            for ann_data in json_data['Annotation']:
                gt_labels.append(CLASSES_dict[ann_data['class_name']])
                gt_bboxes.append(ann_data['data'])


            data_anno = dict(
                    bboxes=np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
                    labels=np.array(gt_labels, dtype=np.long))


            data_info.update(ann=data_anno)
            
            data_infos.append(data_info)
            
            if idx!=0 and idx%20000==0:
                print(str(idx)+'/'+str(len(ls))+' load annotations END!')
            
        
        
        return data_infos

In [4]:
## 추가수정 기존 받았던 pretrain과 매칭되는 config로 수정 
cfg = Config.fromfile('UniverseNet/configs/waymo_open/universenet50_2008_fp16_4x4_mstrain_640_1280_1x_waymo_open_f0.py') 

In [5]:
print(f'Config:\n{cfg.pretty_text}')

Config:
pretrained_ckpt = 'https://github.com/shinya7y/weights/releases/download/v1.0.2/res2net50_v1b_26w_4s-3cf99910_mmdetv2-92ed3313.pth'
model = dict(
    type='GFL',
    backbone=dict(
        type='Res2Net',
        depth=50,
        scales=4,
        base_width=26,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='SyncBN', requires_grad=True),
        norm_eval=False,
        style='pytorch',
        dcn=dict(type='DCN', deform_groups=1, fallback_on_stride=False),
        stage_with_dcn=(False, False, False, True),
        init_cfg=dict(
            type='Pretrained',
            checkpoint=
            'https://github.com/shinya7y/weights/releases/download/v1.0.2/res2net50_v1b_26w_4s-3cf99910_mmdetv2-92ed3313.pth'
        )),
    neck=[
        dict(
            type='FPN',
            in_channels=[256, 512, 1024, 2048],
            out_channels=256,
            start_level=1,
            add_extra_convs='on_output',
   

In [6]:
## 추가 및 수정 ## 
cfg.dataset_type  = 'Drive_dataset'
cfg.data_root = ''

## single GPU 이기 때문에 syncBN 이 아닌 BN으로 수정)
cfg.model.backbone.norm_cfg=dict(type='BN', requires_grad=True)

## Validation pipeline에 train pipeline 적용하기 위해서 구성 
cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='Resize',
        img_scale=(1920, 1200),
        multiscale_mode='range',
        keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.0),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
    
]

### test pipeline 나중에 test진행에 사용할 거 실제 validation은 위의 pipeline 으로 진행
cfg.test_pipeline = [
    ### TSET때 사용할 test time augmentation용 pipeline
    dict(type='LoadImageFromFile'),
    dict(
                type='MultiScaleFlipAug',
                img_scale=(1920, 1200),
                flip=False,
                transforms=[
                    dict(type='Resize', keep_ratio=True),
                    dict(type='RandomFlip'),
                    dict(
                        type='Normalize',
                        mean=[123.675, 116.28, 103.53],
                        std=[58.395, 57.12, 57.375],
                        to_rgb=True),
                    dict(type='Pad', size_divisor=32),
                    dict(type='ImageToTensor', keys=['img']),
                      dict(type='Collect', keys=['img'])
                ])
]

cfg.data=dict(
    samples_per_gpu=10,
    workers_per_gpu=12,
    train=dict(
        type=cfg.dataset_type,
        ann_file='2DBB/new_train.csv',
        pipeline=cfg.train_pipeline),
     val=dict(
        type=cfg.dataset_type,
        ann_file='2DBB/new_valid.csv',
        pipeline=cfg.test_pipeline),
    test=None)

cfg.model.bbox_head.num_classes=10

cfg.device='cuda'
cfg.work_dir = 'checkpoints_ver3'

cfg.log_config.interval = 8000 #iteration 단위

cfg.seed = 2024

set_random_seed(cfg.seed, deterministic=False)

cfg.workflow = [('train', 1), ('val',1)]

cfg.evaluation = dict(interval=1, metric='mAP')

cfg.load_from = 'universenet50_2008_fp16_4x4_mstrain_480_960_2x_coco_20200815_epoch_24-81356447.pth'
cfg.runner = dict(type='EpochBasedRunner', max_epochs=50)

cfg.gpu_ids = range(1)

In [7]:
print(f'Config:\n{cfg.pretty_text}')

Config:
pretrained_ckpt = 'https://github.com/shinya7y/weights/releases/download/v1.0.2/res2net50_v1b_26w_4s-3cf99910_mmdetv2-92ed3313.pth'
model = dict(
    type='GFL',
    backbone=dict(
        type='Res2Net',
        depth=50,
        scales=4,
        base_width=26,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=False,
        style='pytorch',
        dcn=dict(type='DCN', deform_groups=1, fallback_on_stride=False),
        stage_with_dcn=(False, False, False, True),
        init_cfg=dict(
            type='Pretrained',
            checkpoint=
            'https://github.com/shinya7y/weights/releases/download/v1.0.2/res2net50_v1b_26w_4s-3cf99910_mmdetv2-92ed3313.pth'
        )),
    neck=[
        dict(
            type='FPN',
            in_channels=[256, 512, 1024, 2048],
            out_channels=256,
            start_level=1,
            add_extra_convs='on_output',
       

In [8]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
import torch
import copy

In [ ]:
# Build dataset
### validation ###
val_dataset=copy.deepcopy(cfg.data.val)
val_dataset.pipeline=cfg.data.train.pipeline
val_ds = build_dataset(val_dataset)
### validation  ###


## 실제 augmentation 포함 pipeline
cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='Resize',
        img_scale=[(960, 640), (2880, 1800)],
        multiscale_mode='range',
        keep_ratio=True),
    dict(
        type='PhotoMetricDistortion',
        brightness_delta=32,
        contrast_range=(0.5, 1.5),
        saturation_range=(0.5, 1.5),
        hue_delta=18),
     dict(
        type='RandomCrop',
        crop_type='absolute_range',
        crop_size=(640, 960)),  ##### crop size는 (height, width) 로 되있음
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
    
]

cfg.data.train.pipeline = cfg.train_pipeline

### class imbalance 를 해소하기 위해 oversampling 진행 
cfg.data.train = dict(
        type='ClassBalancedDataset',
        oversample_thr=0.1,
        dataset=cfg.data.train
    )

datasets = [build_dataset(cfg.data.train), val_ds]

print(len(datasets[0]))
# Build the detector
model = build_detector(cfg.model)

# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

meta=dict()
meta['config'] = cfg.pretty_text
meta['seed'] = cfg.seed
# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True, meta=meta)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:43: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/root/UniverseNet/mmdet/datasets/custom.py:186: UserWarning: CustomDataset does not support filtering empty gt images.
  'CustomDataset does not support filtering empty gt images.')


20000/80000 load annotations END!
40000/80000 load annotations END!
60000/80000 load annotations END!
95223


/root/UniverseNet/mmdet/models/dense_heads/anchor_head.py:116: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '
2024-08-07 08:46:07,443 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2024-08-07 08:46:09,096 - mmdet - INFO - load checkpoint from local path: universenet50_2008_fp16_4x4_mstrain_480_960_2x_coco_20200815_epoch_24-81356447.pth
2024-08-07 08:46:09,152 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for bbox_head.gfl_cls.weight: copying a param with shape torch.Size([80, 256, 3, 3]) from checkpoint, the shape in current model is torch.Size([10, 256, 3, 3]).
size mismatch for bbox_head.gfl_cls.bias: copying a param with shape torch.Size([80]) from checkpoint, the shape in current model is torch.Size([10]).
2024-08-07 08:46:09,154 - mmdet - INFO - Start running, host: root@de6c2

[>>>>>>>>>>>>>>>>>>>>>>>>] 10000/10000, 15.6 task/s, elapsed: 640s, ETA:     0s
---------------iou_thr: 0.5---------------


2024-08-07 10:49:08,415 - mmdet - INFO - 
+-------------------+-------+--------+--------+-------+
| class             | gts   | dets   | recall | ap    |
+-------------------+-------+--------+--------+-------+
| car               | 32445 | 391072 | 0.974  | 0.915 |
| bus               | 952   | 23451  | 0.935  | 0.821 |
| truck             | 5896  | 139812 | 0.947  | 0.794 |
| special vehicle   | 162   | 29397  | 0.759  | 0.257 |
| motorcycle        | 213   | 9957   | 0.831  | 0.636 |
| bicycle           | 81    | 2635   | 0.790  | 0.572 |
| personal mobility | 45    | 7241   | 0.911  | 0.772 |
| person            | 6890  | 55571  | 0.854  | 0.743 |
| Traffic_light     | 3932  | 32858  | 0.958  | 0.893 |
| Traffic_sign      | 2720  | 93391  | 0.935  | 0.769 |
+-------------------+-------+--------+--------+-------+
| mAP               |       |        |        | 0.717 |
+-------------------+-------+--------+--------+-------+
2024-08-07 10:49:08,480 - mmdet - INFO - Epoch(val) [1][10000]

...

2024-08-09 05:23:21,033 - mmdet - INFO - 
+-------------------+-------+--------+--------+-------+
| class             | gts   | dets   | recall | ap    |
+-------------------+-------+--------+--------+-------+
| car               | 32445 | 238529 | 0.982  | 0.952 |
| bus               | 952   | 9965   | 0.967  | 0.926 |
| truck             | 5896  | 70989  | 0.969  | 0.905 |
| special vehicle   | 162   | 5232   | 0.858  | 0.686 |
| motorcycle        | 213   | 5539   | 0.915  | 0.808 |
| bicycle           | 81    | 967    | 0.963  | 0.896 |
| personal mobility | 45    | 1905   | 0.911  | 0.871 |
| person            | 6890  | 55206  | 0.924  | 0.846 |
| Traffic_light     | 3932  | 20525  | 0.980  | 0.944 |
| Traffic_sign      | 2720  | 26484  | 0.954  | 0.879 |
+-------------------+-------+--------+--------+-------+
| mAP               |       |        |        | 0.871 |
+-------------------+-------+--------+--------+-------+
2024-08-09 05:23:21,041 - mmdet - INFO - Epoch(val) [20][10000

[>>>>>>>>>>>>>>>>>>>>>>>>] 10000/10000, 15.4 task/s, elapsed: 648s, ETA:     0s
---------------iou_thr: 0.5---------------


2024-08-09 07:37:36,006 - mmdet - INFO - 
+-------------------+-------+--------+--------+-------+
| class             | gts   | dets   | recall | ap    |
+-------------------+-------+--------+--------+-------+
| car               | 32445 | 244659 | 0.983  | 0.952 |
| bus               | 952   | 10331  | 0.971  | 0.927 |
| truck             | 5896  | 76058  | 0.970  | 0.904 |
| special vehicle   | 162   | 5521   | 0.870  | 0.697 |
| motorcycle        | 213   | 5678   | 0.911  | 0.806 |
| bicycle           | 81    | 1034   | 0.963  | 0.906 |
| personal mobility | 45    | 1925   | 0.911  | 0.876 |
| person            | 6890  | 55022  | 0.923  | 0.845 |
| Traffic_light     | 3932  | 21298  | 0.980  | 0.944 |
| Traffic_sign      | 2720  | 28824  | 0.956  | 0.880 |
+-------------------+-------+--------+--------+-------+
| mAP               |       |        |        | 0.874 |
+-------------------+-------+--------+--------+-------+
2024-08-09 07:37:36,014 - mmdet - INFO - Epoch(val) [21][10000

[>>>>>>>>>>>>>>>>>>>>>>>>] 10000/10000, 15.4 task/s, elapsed: 649s, ETA:     0s
---------------iou_thr: 0.5---------------


2024-08-09 09:51:54,210 - mmdet - INFO - 
+-------------------+-------+--------+--------+-------+
| class             | gts   | dets   | recall | ap    |
+-------------------+-------+--------+--------+-------+
| car               | 32445 | 233489 | 0.982  | 0.952 |
| bus               | 952   | 9939   | 0.968  | 0.926 |
| truck             | 5896  | 74192  | 0.970  | 0.905 |
| special vehicle   | 162   | 4858   | 0.858  | 0.685 |
| motorcycle        | 213   | 5279   | 0.906  | 0.808 |
| bicycle           | 81    | 997    | 0.963  | 0.882 |
| personal mobility | 45    | 1947   | 0.911  | 0.870 |
| person            | 6890  | 54386  | 0.922  | 0.845 |
| Traffic_light     | 3932  | 19819  | 0.980  | 0.943 |
| Traffic_sign      | 2720  | 26918  | 0.954  | 0.879 |
+-------------------+-------+--------+--------+-------+
| mAP               |       |        |        | 0.870 |
+-------------------+-------+--------+--------+-------+
2024-08-09 09:51:54,218 - mmdet - INFO - Epoch(val) [22][10000